In [ ]:
import pandas as pd
import vaex

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy - 20 for soil
# 1924 earliest year

In [ ]:
# read in gbif_init
df_path = input("Enter the file path: ")

# df = vaex.open(df_path)
df = pd.read_parquet(df_path)

In [ ]:
df

In [ ]:
# remove rows where coordinateUncertaintyInMeters is > 20
df = df[df["coordinateUncertaintyInMeters"] <= 20]

df

In [ ]:
# double check earliest year of record
year_unique = df["year"].unique()

In [ ]:
coordinate_unique = df["coordinateUncertaintyInMeters"].unique()

In [ ]:
# remove values that aren't numbers
df = df[df.coordinateUncertaintyInMeters != "da86174a-a605-43a4-a5e8-53d484152cd3"]

In [ ]:
df = df[df.coordinateUncertaintyInMeters != "2023-07-06T12:41:54.635Z"]

In [ ]:
# convert column coordinateUncertaintyInMeters to float
df["coordinateUncertaintyInMeters"] = df["coordinateUncertaintyInMeters"].astype(
    "float64"
)

In [ ]:
# find the highest number in the coordinateUncertaintyInMeters column using vaex.max and store the result in a new variable
max_uncertainty_in_meters = df["coordinateUncertaintyInMeters"].max()

max_uncertainty_in_meters

In [ ]:
min_uncertainty_in_meters = df["coordinateUncertaintyInMeters"].min()

min_uncertainty_in_meters

In [ ]:
# count the number of na values in column species
species_na = df["species"].isna().sum()

latitude_na = df["decimalLatitude"].isna().sum()

longitude_na = df["decimalLongitude"].isna().sum()

coordinate_na = df["coordinateUncertaintyInMeters"].isna().sum()

print(species_na, latitude_na, longitude_na, coordinate_na)

In [ ]:
df = df.dropna(
    subset=[
        "species",
        "decimalLatitude",
        "decimalLongitude",
        "coordinateUncertaintyInMeters",
        "year",
    ]
)

In [ ]:
species_na = df["species"].isna().sum()

latitude_na = df["decimalLatitude"].isna().sum()

longitude_na = df["decimalLongitude"].isna().sum()

coordinate_na = df["coordinateUncertaintyInMeters"].isna().sum()

print(species_na, latitude_na, longitude_na, coordinate_na)

In [ ]:
df["decimalLatitude"] = df["decimalLatitude"].astype("float64")

In [ ]:
max_lat = df["decimalLatitude"].max()

min_lat = df["decimalLatitude"].min()

print(max_lat, min_lat)

In [ ]:
df["decimalLongitude"] = df["decimalLongitude"].astype("float64")

In [ ]:
max_long = df["decimalLongitude"].max()

min_long = df["decimalLongitude"].min()

print(max_long, min_long)

In [ ]:
# print value of rightsHolder in first row
df["rightsHolder"].iloc[0]

In [ ]:
df["rights"] = None

df

In [ ]:
rights_holder_unique = df["rightsHolder"].unique()

print(rights_holder_unique)

In [ ]:
# for i in range(len(df)):
#     if df['rightsHolder'].iloc[i] == None:
#         df['rights'].iloc[i] = None
#     else:
#         if df['license'].iloc[i] == None:
#             df['rights'].iloc[i] = df['rightsHolder'].iloc[i]
#         else:
#             df['rights'].iloc[i] = df['rightsHolder'].iloc[i] + ',' + df['license'].iloc[i]


def merge_columns(row):
    if pd.isnull(row["rightsHolder"]):
        return row["rightsHolder"]

    else:
        return row["rightsHolder"] + ", " + row["license"]


# apply the function to the dataframe
df["rights"] = df.apply(merge_columns, axis=1)

df

In [ ]:
rights_unique = df["rights"].unique()

rights_unique

In [ ]:
# create a new variable that only contains the columns kingdom, phylum, class, order, family, genus, species, decimalLatitude, decimalLongitude, license, and rightsHolder
df_essential = df[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "species",
        "decimalLatitude",
        "decimalLongitude",
        "rights",
    ]
]

df_essential

In [ ]:
# read df_essential into a parquet file where user can input the file path
df_essential_path = input("Enter the file path: ")
df_essential.export_parquet(df_essential_path)